In [1]:
%load_ext autoreload
%autoreload 2

In [19]:
import sys
from tqdm import tqdm
import pickle
sys.path.append('../../')
from ocr_ensemble.postprocessing import RobertaPostprocessor, RobertaPreLNPostprocessor, XLMRobertaPostprocessor
from ocr_ensemble.evaluation import average_levenshtein
from weighted_levenshtein import lev


In [6]:
with open('../../data/label_studio/labels.pkl', 'rb') as f:
    labels = pickle.load(f)

In [7]:
with open('../../data/label_studio/paddle_predictions.pkl', 'rb') as f:
    paddle = pickle.load(f)

In [8]:
len(paddle)

2290

In [9]:
len(labels)

2290

In [37]:
post = RobertaPostprocessor(size='base')
print(post(['texttisbetter', 'BECOMINGMORE']))

['text=better', 'BECOMING MORE']


In [38]:
batch_size = 50
batch = []
predictions_roberta = []
for pred in tqdm(paddle):
    batch += [pred]
    if len(batch) % batch_size == 0:
        curr = post(batch, filter=True)
        predictions_roberta += curr
        batch = []
if len(batch) > 0:
    curr = post(batch, filter=True)
    predictions_roberta += curr
    batch = []

100%|████████████████████████████████████████████████████████████████████████████| 2290/2290 [00:02<00:00, 1109.64it/s]


In [39]:
average_levenshtein(labels, predictions_roberta)

1.9899563318777294

In [40]:
average_levenshtein(predictions_roberta, paddle)

0.36899563318777295

In [17]:
with open('../../data/label_studio/chatgpt35_postprocessing.pkl', 'rb') as f:
    chatgpt = pickle.load(f)

In [18]:
average_levenshtein(labels, chatgpt)

3.435807860262009

In [21]:
chatgpt_filtered = []
for idx, (p, pp) in enumerate(zip(paddle, chatgpt)):
    if lev(pp.encode("ascii", "ignore").decode(), p.encode("ascii", "ignore").decode()) > 5:
        chatgpt_filtered.append(p)
    else:
        chatgpt_filtered.append(pp)

In [22]:
average_levenshtein(labels, chatgpt_filtered)

1.9685589519650655

In [23]:
average_levenshtein(paddle, chatgpt_filtered)

0.5397379912663756

In [24]:
average_levenshtein(paddle, chatgpt)

2.3799126637554586